In [5]:
import pandas as pd
import numpy as np

In [95]:
import sqlite3
import csv

# Function to connect to a .db file and export the contents of a specific table to a CSV file
def export_table_to_csv(db_path, table_name, csv_filename):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Retrieve and print the names of all tables in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        print("Tables in the database:", [table[0] for table in tables])
        
        # Fetch the content of the specified table
        cursor.execute(f"SELECT * FROM {table_name};")
        rows = cursor.fetchall()
        
        # Fetch column names
        column_names = [description[0] for description in cursor.description]
        
        # Write the content to a CSV file
        with open(csv_filename, mode='w', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerow(column_names)  # Write header row
            writer.writerows(rows)         # Write data rows
        
        print(f"Table '{table_name}' exported successfully to '{csv_filename}'")
        
    except sqlite3.Error as e:
        print(f"Error loading .db file: {e}")
    finally:
        # Close the connection
        if conn:
            conn.close()

P = "/home/sachin/prompting_attack/PromptAttack_LLM/"
# Replace the following with your actual file paths and table name
name = "attack_4_new"
export_table_to_csv(f'results/results/{name}.db', 'my_table', f'results/results/output_csvs/{name}_output.csv')


Tables in the database: ['my_table']
Table 'my_table' exported successfully to 'results/results/output_csvs/attack_6_new_output.csv'


In [96]:
df = pd.read_csv('results/results/output_csvs/attack_6_new_output.csv')

In [97]:
def get_sent(long_text):
    # Extracting the sentence
    start = long_text.find('"') + 1  # Find the first quote
    end = long_text.find('"', start)  # Find the closing quote
    original_sentence = long_text[start:end]
    
    return original_sentence

paraphrase_texts = []
original_texts = []
for index, row in df.iterrows():
    text = row['V']
    original_text = get_sent(row['Q']) #row['Q'].split('\n')[-1].split('->')[0]
    texts_postprocess = text.split('->')[1].strip().split('\n')
    if(len(texts_postprocess)==1):
        texts_postprocess = text.split('->')[1].strip().split('  ')[0]
    else:
        texts_postprocess = texts_postprocess[0]
    # print(texts_postprocess)
    # print('##########')
    paraphrase_texts.append(texts_postprocess)
    original_texts.append(original_text)
    

In [98]:
df['Text'] = original_texts
df['paraphrase_text'] = paraphrase_texts

In [99]:
val = pd.read_csv('validation_SST.csv')

In [100]:
df_merged = pd.merge(df, val, on="Text")
del df_merged['idx']

In [101]:
df_merged.to_csv('results/results/output_csvs/attack_6_new_output_final.csv', index=False)

## FSR, ASR, RA 

In [260]:
df = pd.read_csv('results/results/output_csvs/attack_4_new_output_final_origpreds_parapreds.csv')
df = df.fillna('')

acnt, fcnt = 0, 0
asr_cnt, ra_cnt, fsr_cnt = 0, 0, 0
for index, row in df.iterrows():
    pred = int(row['original_pred'])
    gold = int(row['label'])
    llama = int(row['pred'])
    gpt4 = int(row['gpt4'])

    #print(index, row['paraphrase_text'])
    paraphrase_text = row['paraphrase_text'].strip()
    #print(paraphrase_text)

    if(gold!=llama and llama==gpt4):
        if(paraphrase_text.lower()==''):
            #print(paraphrase_text)
            acnt+=1
            pass
        else:
            asr_cnt+=1

    if(pred==gold and llama!=gold):
        if(paraphrase_text.lower()==''):
            #print(paraphrase_text)
            fcnt+=1
            #print(paraphrase_text)
            pass
        else:
            fsr_cnt+=1

    if(llama==gold):
        ra_cnt+=1

print(f'FSR: {fsr_cnt/len(df)}')
print(f'ASR: {asr_cnt/len(df)}')
print(f'RA: {ra_cnt/len(df)}')

FSR: 0.6295871559633027
ASR: 0.643348623853211
RA: 0.30619266055045874


In [195]:
fcnt, acnt

(33, 35)

In [262]:
df = pd.read_csv('results/results/output_csvs/attack_6_new_output_final_origpreds_parapreds.csv')
df = df.fillna('')

acnt, fcnt = 0, 0
asr_cnt, ra_cnt, fsr_cnt = 0, 0, 0
for index, row in df.iterrows():
    pred = int(row['original_pred'])
    gold = int(row['label'])
    llama = int(row['pred'])
    gpt4 = int(row['gpt4'])

    paraphrase_text = row['paraphrase_text'].strip()
            

    if(gold!=llama and llama==gpt4):
        if(paraphrase_text.lower()==''):
            #print(paraphrase_text)
            acnt+=1
            pass
        else:
            asr_cnt+=1

    if(pred==gold and llama!=gold):
        if(paraphrase_text.lower()==''):
            #print(paraphrase_text)
            fcnt+=1
            #print(paraphrase_text)
            pass
        else:
            fsr_cnt+=1

    if(llama==gold):
        ra_cnt+=1

print(f'FSR: {fsr_cnt/len(df)}')
print(f'ASR: {asr_cnt/len(df)}')
print(f'RA: {ra_cnt/len(df)}')

FSR: 0.4896788990825688
ASR: 0.481651376146789
RA: 0.44724770642201833


In [198]:
fcnt, acnt

(136, 148)

In [269]:
df = pd.read_csv('results/results/output_csvs/attack_6_new_output_final_origpreds_parapreds.csv')

asr_cnt, ra_cnt, fsr_cnt = 0, 0, 0
for index, row in df.iterrows():
    # pred = int(row['pred_original'])
    gold = int(row['label'])
    llama = int(row['pred'])
    gpt4 = int(row['gpt4'])

    if(gold!=llama and llama==gpt4):
        asr_cnt+=1

    if(pred==gold and llama!=gold):
        fsr_cnt+=1

    if(llama==gold):
        ra_cnt+=1

print(f'FSR: {fsr_cnt/len(df)}')
print(f'ASR: {asr_cnt/len(df)}')
print(f'RA: {ra_cnt/len(df)}')

FSR: 0.2408256880733945
ASR: 0.3486238532110092
RA: 0.5837155963302753


In [186]:
df = pd.read_csv('results/output_csvs_7B/attack_8_output_final_gpt4_llama_preds_updated.csv')
df.fillna('')
df['paraphrase_text'].value_counts()

paraphrase_text
the far future may be awesome to consider, but from period detail to matters of the heart, this film is most transporting when it stays put in the recent.    1
unflinchingly hopeful and determined                                                                                                                          1
it's slow -- very slow, very slow.                                                                                                                            1
a captivating film at times tedious.                                                                                                                          1
it's a dull and uninspiring trip.                                                                                                                             1
                                                                                                                                                             ..
the iditarod lasts for d

In [279]:
df = pd.read_csv('results/output_csvs_7B/attack_9_output_final_gpt4_preds.csv') #attack_3_output_final_gpt4_llama_preds_updated.csv')
df = df.fillna('')

cnt=0
for index, row in df.iterrows():
    # pred = int(row['original_pred'])
    gold = int(row['label'])
    llama = int(row['pred'])
    gpt4 = int(row['gpt4'])

    if(llama!=gpt4):
        cnt+=1


print(f'FSR: {cnt/len(df)}')
# print(f'ASR: {asr_cnt/len(df)}')
# print(f'RA: {ra_cnt/len(df)}')

FSR: 0.10091743119266056


In [143]:
## ASR, AAA
import pandas as pd

In [144]:
df = pd.read_csv('results/results_old/output_csvs_7B/attack_1_output_final_gpt4_llama_preds_updated1.csv')
df = df.fillna('')

In [146]:
orig = list(df['original_pred'])
label = list(df['label'])
pred = list(df['pred'])

In [147]:
def get_AfterAttackAccuracy(pred, adv_pred, label):
    assert len(pred) == len(label) and len(pred) == len(adv_pred)
    adv_wrong = [
        (pred[i] == label[i] and adv_pred[i] == label[i]) for i in range(len(label))
    ]
    return sum(adv_wrong) / len(label)

def get_BeforeAttackAccuracy(pred, label):
    assert len(pred) == len(label)
    adv_wrong = [
        (pred[i] == label[i]) for i in range(len(label))
    ]
    return sum(adv_wrong) / len(label)

def get_ASR(pred, adv_pred, label):
    assert len(pred) == len(label) and len(pred) == len(adv_pred)
    correct = [i == j for (i, j) in zip(pred, label)]
    adv_wrong = [
        (pred[i] == label[i] and adv_pred[i] != label[i]) for i in range(len(label))
    ]
    return sum(adv_wrong) / len(correct)

In [140]:
acc = get_AfterAttackAccuracy(orig, pred, label)

In [148]:
baa = get_BeforeAttackAccuracy(orig, label)

In [141]:
asr = get_ASR(orig, pred, label)

In [149]:
print(f'After Attack Accuracy: {acc}')
print(f'Before Attack Accuracy: {baa}')
print(f'Attack Success Rate: {asr}')

After Attack Accuracy: 0.2511467889908257
Before Attack Accuracy: 0.9231651376146789
Attack Success Rate: 0.6720183486238532


In [3]:
import pandas as pd

In [4]:
df1 = pd.read_csv('results/results/output_csvs/attack_4_new_output_final.csv')
df2 = pd.read_csv('results/results/output_csvs/attack_6_new_output_final.csv')

In [6]:
df1

,Q,V,Text,paraphrase_text,label
0,"The original sentence ""a sometimes tedious fil...","The original sentence ""a sometimes tedious fil...",a sometimes tedious film .,a sometimes engrossing film.,0
1,"The original sentence ""unflinchingly bleak and...","The original sentence ""unflinchingly bleak and...",unflinchingly bleak and desperate,unflinchingly hopeful and optimistic,0
2,"The original sentence ""it 's slow -- very , ve...","The original sentence ""it 's slow -- very , ve...","it 's slow -- very , very slow .","it's slow -- very, very fast.",0
3,"The original sentence ""holden caulfield did it...","The original sentence ""holden caulfield did it...",holden caulfield did it better .,holden caulfield did it worse.,0
4,"The original sentence ""i had to look away - th...","The original sentence ""i had to look away - th...",i had to look away - this was god awful .,i had to look away - this was god awesome.,0
...,...,...,...,...,...
867,"The original sentence ""... is an arthritic att...","The original sentence ""... is an arthritic att...",... is an arthritic attempt at directing by ca...,... is an arthritic attempt at directing by kh...,0
868,"The original sentence ""a movie with a real ana...","The original sentence ""a movie with a real ana...",a movie with a real anarchic flair .,a movie with a flair anarchic real.,1
869,"The original sentence ""the far future may be a...","The original sentence ""the far future may be a...","the far future may be awesome to consider , bu...","the far future may be awesome to consider, but...",1
870,"The original sentence ""mcconaughey 's fun to w...","The original sentence ""mcconaughey 's fun to w...","mcconaughey 's fun to watch , the dragons are ...","mcconaughey's fun to watch, the dragons are ok...",1


In [7]:
df2

,Q,V,Text,paraphrase_text,label
0,"The original sentence ""holden caulfield did it...","The original sentence ""holden caulfield did it...",holden caulfield did it better .,holden caulfield did it worse.,0
1,"The original sentence ""... the movie is just a...","The original sentence ""... the movie is just a...",... the movie is just a plain old monster .,... the monster is just a plain old movie.,0
2,"The original sentence ""it 's slow -- very , ve...","The original sentence ""it 's slow -- very , ve...","it 's slow -- very , very slow .","it's very slow -- very, very slow.",0
3,"The original sentence ""it 's a charming and of...","The original sentence ""it 's a charming and of...",it 's a charming and often affecting journey .,it's a charming and often depressing journey.,1
4,"The original sentence ""it 's a cookie-cutter m...","The original sentence ""it 's a cookie-cutter m...","it 's a cookie-cutter movie , a cut-and-paste ...","it's a cut-and-paste movie, a cookie-cutter job.",0
...,...,...,...,...,...
867,"The original sentence ""a subject like this sho...","The original sentence ""a subject like this sho...",a subject like this should inspire reaction in...,a subject like this should inspire reaction in...,0
868,"The original sentence ""mcconaughey 's fun to w...","The original sentence ""mcconaughey 's fun to w...","mcconaughey 's fun to watch , the dragons are ...","mcconaughey's fun to watch, the dragons are ok...",1
869,"The original sentence ""looking aristocratic , ...","The original sentence ""looking aristocratic , ...","looking aristocratic , luminous yet careworn i...","looking aristocratic, luminous yet careworn in...",1
870,"The original sentence ""the far future may be a...","The original sentence ""the far future may be a...","the far future may be awesome to consider , bu...","the far future may be awesome to consider, but...",1


In [8]:
df_merged = pd.merge(df1,df2,on='Text')
df_merged

,Q_x,V_x,Text,paraphrase_text_x,label_x,Q_y,V_y,paraphrase_text_y,label_y
0,"The original sentence ""a sometimes tedious fil...","The original sentence ""a sometimes tedious fil...",a sometimes tedious film .,a sometimes engrossing film.,0,"The original sentence ""a sometimes tedious fil...","The original sentence ""a sometimes tedious fil...",a sometimes boring film. (positive),0
1,"The original sentence ""unflinchingly bleak and...","The original sentence ""unflinchingly bleak and...",unflinchingly bleak and desperate,unflinchingly hopeful and optimistic,0,"The original sentence ""unflinchingly bleak and...","The original sentence ""unflinchingly bleak and...",unflinchingly hopeful and determined,0
2,"The original sentence ""it 's slow -- very , ve...","The original sentence ""it 's slow -- very , ve...","it 's slow -- very , very slow .","it's slow -- very, very fast.",0,"The original sentence ""it 's slow -- very , ve...","The original sentence ""it 's slow -- very , ve...","it's very slow -- very, very slow.",0
3,"The original sentence ""holden caulfield did it...","The original sentence ""holden caulfield did it...",holden caulfield did it better .,holden caulfield did it worse.,0,"The original sentence ""holden caulfield did it...","The original sentence ""holden caulfield did it...",holden caulfield did it worse.,0
4,"The original sentence ""i had to look away - th...","The original sentence ""i had to look away - th...",i had to look away - this was god awful .,i had to look away - this was god awesome.,0,"The original sentence ""i had to look away - th...","The original sentence ""i had to look away - th...",i had to look away - this was god awful.,0
...,...,...,...,...,...,...,...,...,...
867,"The original sentence ""... is an arthritic att...","The original sentence ""... is an arthritic att...",... is an arthritic attempt at directing by ca...,... is an arthritic attempt at directing by kh...,0,"The original sentence ""... is an arthritic att...","The original sentence ""... is an arthritic att...",... is an arthritic attempt at directing by ca...,0
868,"The original sentence ""a movie with a real ana...","The original sentence ""a movie with a real ana...",a movie with a real anarchic flair .,a movie with a flair anarchic real.,1,"The original sentence ""a movie with a real ana...","The original sentence ""a movie with a real ana...",a movie with a real anarchic flare.,1
869,"The original sentence ""the far future may be a...","The original sentence ""the far future may be a...","the far future may be awesome to consider , bu...","the far future may be awesome to consider, but...",1,"The original sentence ""the far future may be a...","The original sentence ""the far future may be a...","the far future may be awesome to consider, but...",1
870,"The original sentence ""mcconaughey 's fun to w...","The original sentence ""mcconaughey 's fun to w...","mcconaughey 's fun to watch , the dragons are ...","mcconaughey's fun to watch, the dragons are ok...",1,"The original sentence ""mcconaughey 's fun to w...","The original sentence ""mcconaughey 's fun to w...","mcconaughey's fun to watch, the dragons are ok...",1


In [13]:
cnt=0
for i in range(len(df_merged)):
    if(df_merged.iloc[i]['paraphrase_text_x'] == df_merged.iloc[i]['paraphrase_text_y']):
        cnt+=1
cnt

94